In [2]:
# 📌 Image Colorization using OpenCV and Pre-trained CNN
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the pre-trained CNN and cluster centers
prototxt = "model/colorization_deploy_v2.prototxt"
model = "model/colorization_release_v2.caffemodel"
points = "model/pts_in_hull.npy"

# Load the neural network
net = cv2.dnn.readNetFromCaffe(prototxt, model)
pts = np.load(points)

# Load the cluster centers into the model
class8 = net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")
pts = pts.transpose().reshape(2, 313, 1, 1)
net.getLayer(class8).blobs = [pts.astype("float32")]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype="float32")]

# Read a grayscale image
image_path = "images/sample_bw.jpg"
image = cv2.imread(image_path)
scaled = image.astype("float32") / 255.0

# Convert to Lab and preprocess
lab = cv2.cvtColor(scaled, cv2.COLOR_BGR2LAB)
resized = cv2.resize(lab, (224, 224))
L = cv2.split(resized)[0]
L -= 50  # Mean-centering

# Predict ab channels
net.setInput(cv2.dnn.blobFromImage(L))
ab = net.forward()[0, :, :, :].transpose((1, 2, 0))
ab = cv2.resize(ab, (image.shape[1], image.shape[0]))

# Combine with L channel
L_original = cv2.split(lab)[0]
colorized = np.concatenate((L_original[:, :, np.newaxis], ab), axis=2)
colorized = cv2.cvtColor(colorized, cv2.COLOR_LAB2BGR)
colorized = np.clip(colorized, 0, 1)

# Show results
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title("Input Grayscale")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(colorized)
plt.title("Colorized Output")
plt.axis('off')
plt.show()


ModuleNotFoundError: No module named 'cv2'